In [5]:
# generic import
import numpy as np
import itertools as iter
import os

# mne import
import mne
from mne import Epochs, pick_types, events_from_annotations
from mne.io import concatenate_raws
from mne.io.edf import read_raw_edf
from mne.datasets import eegbci
from mne import set_log_level

set_log_level(verbose=False)

In [1]:
# generic import
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import itertools as iter
import scipy as sc
import pickle as pk

# mne import
import mne
from mne import Epochs, pick_types, events_from_annotations
from mne.io import concatenate_raws
from mne.io.edf import read_raw_edf
from mne.datasets import eegbci
from mne import set_log_level

# pyriemann import
import pyriemann
from pyriemann.classification import MDM, TSclassifier, class_distinctiveness
from pyriemann.estimation import Covariances, Coherences
from pyriemann.spatialfilters import CSP
from pyriemann.tangentspace import TangentSpace
from pyriemann.embedding import SpectralEmbedding
from pyriemann.transfer import encode_domains, decode_domains, TLCenter, TLStretch, TLRotate
from pyriemann.utils.mean import mean_covariance
from pyriemann.clustering import Kmeans

# sklearn imports
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import silhouette_score, davies_bouldin_score, accuracy_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, TimeSeriesSplit, BaseCrossValidator
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

import itertools

set_log_level(verbose=False)

In [2]:
selected_events = None
inconsistent_session = False

# ========================

from moabb.datasets import Schirrmeister2017

# ====================================
# ======== Schirrmeister2017 =========
# ====================================

dataset1 = Schirrmeister2017()
subjects = [i+1 for i in range(14)] # 14
sessions = ["0"]
runs = ["0train", "1test"]
tmin, tmax = 1., 3.
sample_step = 3 # chan = 128
dispersion_1 = 50
dispersion_2 = 1000
dispersion_3 = 100

# ====================================

n_subjects = len(subjects)
n_sessions = len(sessions)
n_runs = len(runs)

# ========================

<frozen importlib._bootstrap>:219: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:219: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:219: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
/home/cameronboruto/.local/lib/python3.8/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.
Loading subjects [1, 2]...


In [2]:
all_epochs = []
all_labels = []

for subject in subjects:
    print(f"Loading subjects {subjects}...")
    data = dataset1.get_data(subjects=[subject])

    print(f"Processing raw data for subject {subject}")
    # ========================
    
    raw_files = [
        data[subject][ses][run] for ses, run in iter.product(sessions, runs)
    ]
    raw = concatenate_raws(raw_files)
        
    picks = pick_types(
        raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')
    # subsample elecs
    picks = picks[::sample_step]

    # Apply band-pass filter
    raw.filter(7., 35., method='iir', picks=picks)

    events, event_ids = mne.events_from_annotations(raw, event_id = selected_events)

    # Reannotation
    # Get the annotations
    annotations = raw.annotations.copy()

    # Loop through annotations to find trial onsets
    for idx, annot in enumerate(raw.annotations):  # Iterate until the second to last annotation
        # Create a new annotation for resting-state
        if selected_events:
            if raw.annotations[idx]['description'] not in selected_events:
                continue
        if raw.annotations[idx]['duration'] > 0:
            duration = raw.annotations[idx]['duration']
            duration = max(1, duration)
            onset = annot['onset'] - duration
            description = 'idle'

            # Add this resting-state period as a new annotation
            annotations.append(onset, duration, description)

    # Update the annotations in the raw data
    raw.set_annotations(annotations)

    # Extract events including resting-state periods
    if selected_events:
        events, event_ids = mne.events_from_annotations(raw)
        selected_events['idle'] = event_ids['idle']
    events, event_ids = mne.events_from_annotations(raw, event_id = selected_events)
    
    # ========================

    # Read epochs (train will be done only between 1 and 2s)
    # Testing will be done with a running classifier
    epochs = Epochs(
        raw,
        events,
        None,
        tmin,
        tmax,
        proj=True,
        picks=picks,
        baseline=None,
        preload=True,
        verbose=False)
    labels = epochs.events[:, -1]

    print("Shape of events: ", events.shape[0])
    print("Shape of labels: ", labels.shape[0])
    
    all_epochs.append(epochs)
    all_labels.append(labels)

    os.rmdir("~/mne_data/")

<frozen importlib._bootstrap>:219: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:219: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:219: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
/home/cameronboruto/.local/lib/python3.8/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.
Loading subjects [1, 2]...


In [7]:
epoch_path = "../epoch_data"

# Check if the directory already exists
if not os.path.exists(epoch_path):
    # If it doesn't exist, create the directory
    os.mkdir(epoch_path)
    print(f"Directory '{epoch_path}' created successfully.")
else:
    print(f"Directory '{epoch_path}' already exists.")

for i, sub in enumerate(subjects):
    with open(f"{epoch_path}/sch_{sub}_epoch.pkl", 'wb') as f:
        pk.dump(all_epochs[i], f)
    with open(f"{epoch_path}/sch_{sub}_label.pkl", 'wb') as f:
        pk.dump(all_labels[i], f)

print("Save epoched data to epoch_data. Execute mDA_Schirrmeister.ipynb to show the analysis.")

Directory './epoch_data' already exists.
Save epoched data to epoch_data. Execute mDA_Schirrmeister.ipynb to show the analysis.
